In [ ]:
import numpy as np
from numpy.lib import recfunctions as rfn
import secrets
import matplotlib.pyplot as plt

#mpl.rc('font', size=16)
#mpl.rc('text', usetex=True)
#mpl.rc('font', family='serif', serif='palatino')

In [ ]:
# Initializes the Card Class
# Each card has a value and color associated with it
class Card:
  def __init__(self, value, color):
    self.value = str(value)
    self.color = str(color)

#  def returnValue(self):
#    return self.value

#  def returnColor(self):
#    return self.color

In [ ]:
# Initializes the Player Class
# Each player has a hand and number associated with them
class Player:
  def __init__(self, hand, num):
    self.hand = hand
    self.num = num

In [ ]:
def initDeck():
    # creating the 108 card deck with Card objects 
    # shuffles at the end and returns the shuffled array :)
    deck = np.empty(108, dtype=Card)

    colors = ["Red", "Blue", "Green", "Yellow"]
    specials = ["Skip", "Reverse", "Draw2"]
    black_specials = ["Wild", "Draw4"]

    index = 0
    for color in colors:
      i = 0
      while i < 10:
        deck[index] = Card(i, color)
        index += 1
        i += 1
      i = 1
      while i < 10:
        deck[index] = Card(i, color)
        index += 1
        i += 1
      for special in specials:
        deck[index] = Card(special, color)
        index += 1
        deck[index] = Card(special, color)
        index += 1
    
    i = 0
    while i < 4:
      for blk in black_specials:
        deck[index] = Card(blk, "Black")
        index += 1
      i += 1
      

    #print(deck[0].value)    
    np.random.shuffle(deck)
    return deck

In [ ]:
def returnValue(a):
  return a.value

def returnColor(a):
  return a.color

v_value = np.vectorize(returnValue)
v_color = np.vectorize(returnColor)

In [ ]:
def printPlayers(players):
  for p in players:
    print()
    print("Player " + str(p.num) + " has cards: ")
    for c in p.hand:
      print(c.color, c.value, end=", ")
    print()
  return True 

def initGame(N):
  
  # set the input seed
  # generates the starting values for a single Uno game with N players.
  # Will create/shuffle the deck, randomize hands, determine turn order, 
  # determine who starts, and set at least a single wild in Player 1's hand.

  deck = initDeck()
  #wild = Card("Wild", "Black")

  hand1 = np.empty(7, dtype=Card)
  players = np.empty(N, dtype=Player)

  # **** wild start *****
  # removing the wild from the deck that will start in Player 1's hand
  wild_locations = np.array(np.where(v_value(deck) == 'Wild'))
  deck = np.delete(deck, wild_locations[0,0]) 
  hand1[0] = Card("Wild", "Black") # setting the preset card
  # ***************************
  i = 1
  while i < 7:
      card = Card(v_value(deck[i]), v_color(deck[i]))
      hand1[i] = card
      deck = np.delete(deck, i)
      i += 1

  turn_order = np.arange(1,N+1,1)
  np.random.shuffle(turn_order) # why are we shuffling player later if turn order is shuffled here

  players[0] = Player(hand1, 1)

  player_count = 1
  while player_count < N:
    hand = np.empty(7, dtype=Card)
    i = 0
    while i < 7:
      card = Card(v_value(deck[0]), v_color(deck[0]))
      deck = np.delete(deck, 0)
      hand[i] = card
      i += 1
    players[player_count] = Player(hand, player_count+1)
    player_count += 1
  

  np.random.shuffle(players)

  #print("Starting Hands:")
  #printPlayers(players)

  discard = []
  discard.append(Card(v_value(deck[0]), v_color(deck[0])))
  deck = np.delete(deck, 0)

  #print()
  #*************************** FIX IF STARTING CARD WILD
  #print("The starting card is: ", end="")
  #print(discard[0].value, discard[0].color)

  #print()

  #print("Game initialized successfully.")
  return players, deck, discard

In [ ]:

def initGameFixedHands(N, filterWilds = True):
  deck = initDeck()
  #wild = Card("Wild", "Black")

  hand1 = np.empty(7, dtype=Card)
  players = np.empty(N, dtype=Player)

  # **** wild start *****
  # removing the wild from the deck that will start in Player 1's hand
  wild_locations = np.array(np.where(v_value(deck) == 'Wild'))
  deck = np.delete(deck, wild_locations[0,0]) 
  hand1[0] = Card("Wild", "Black") # setting the preset card
  # ***************************
  i = 1
  while i < 7:
    while(v_color(deck[i]) == 'Black'):
      np.random.shuffle(deck)

    card = Card(v_value(deck[i]), v_color(deck[i]))
    hand1[i] = card
    deck = np.delete(deck, i)
    i += 1

  players[0] = Player(hand1, 1)

  player_count = 1
  while player_count < N:
    hand = np.empty(7, dtype=Card)
    i = 0
    while i < 7:
      while(v_color(deck[i]) == 'Black'):
        np.random.shuffle(deck)
      card = Card(v_value(deck[0]), v_color(deck[0]))
      deck = np.delete(deck, 0)
      hand[i] = card
      i += 1
    players[player_count] = Player(hand, player_count+1)
    player_count += 1
  

  np.random.shuffle(players)

  #print("Starting Hands:")
  #printPlayers(players)
  while deck[0].color == 'Black':
    np.random.shuffle(deck)
  discard = []
  discard.append(Card(v_value(deck[0]), v_color(deck[0])))
  deck = np.delete(deck, 0)

  #print()
  #print("The starting card is: ", end="")
  #print(discard[0].value, discard[0].color)

  #print()

  #print("Game initialized successfully.")
  return players, deck, discard

In [ ]:
def findPlayable(hand, curr, wild_played, color_chosen):
  # This function inputs a player's hand and finds all of the playable cards. 
  # it will return two arrays (for now): one with the colored playable cards,
  # and another with all the black cards in hand
  # consider adding later: playable colors & values separately

  playable_colors = []
  black_cards = []

  if wild_played:
    for card in hand: 
      if card.color == "Black":
        black_cards.append(card)
      elif card.color == color_chosen:
        playable_colors.append(card)
  else:
    for card in hand:
      if curr.color == card.color:
        #print("match in color!")
        playable_colors.append(card)
      elif curr.value == card.value:
        #print("match in value!")
        playable_colors.append(card)
      elif card.color == "Black":
        black_cards.append(card)

  np.array(playable_colors)
  np.array(black_cards)
  
  return playable_colors, black_cards


In [ ]:
# general game mechanics functions
def playRandomCard(playable_hand):
  np.random.shuffle(playable_hand)
  return playable_hand[0] # returns random card out of the playable cards

def findIndex(card_check, hand):
  index = 0
  for card in hand:
    if card.value == card_check.value and card.color == card_check.color:
      # if perfect match
      return index
    index += 1
  return -1

def drawCard(hand, deck, discard):
  # ****************** check for if the deck is empty!!!!!!!
  added_card = Card(v_value(deck[0]), v_color(deck[0]))
  deck = np.delete(deck, 0)
  hand_list = list(hand)
  hand_list.append(added_card)
  hand = np.array(hand_list)

  # check for if the deck is empty...
  if len(deck) == 0:
    deck, discard, top_card = shuffleDeck(deck, discard)
    dis_count = 0
    #print("Deck was Empty! Discard pile was shuffled into the deck.")
    discard = discard.tolist()
    discard.append(top_card)

  return added_card, hand, deck, discard

def printCard(card):
  print(card.color, card.value)
  return True

def nextTurn(player_index, rev_incr, N):
  player_index += rev_incr # go to next player
  if player_index < 0:
    player_index = N-1 # go to last player in player list
  elif player_index == N:
    player_index = 0 # go to first player
  return player_index

def shuffleDeck(deck, discard):
  # shuffles the discard into the deck.
  # only called when deck is empty.
  top_card = discard[len(discard) - 1]
  deck = discard
  np.random.shuffle(deck)
  discard = []
  discard = np.array(discard)

  return deck, discard, top_card

In [ ]:
# Action cards functions
def skipNextTurn(player_index, rev_incr, N):
  player_index += rev_incr
  player_index += rev_incr # skip past the next player
  if player_index == -1:
    player_index = N-1 # go to last player in player list
  elif player_index == -2:
    player_index = N-2 # go to second to last player, last got skipped
  elif player_index == N:
    player_index = 0 # go to first player
  elif player_index == N+1:
    player_index = 1 # go to 2nd player, 1st player got skipped
  return player_index

def drawTwo(player_index, rev_incr, N, players, deck, discard):
  index_orig = player_index # saves this so we can use func below

  # gives us the index of the player who needs to draw two cards
  p = players[nextTurn(player_index, rev_incr, N)]
  added_card, p.hand, deck, discard = drawCard(p.hand, deck, discard)
  added_card, p.hand, deck, discard = drawCard(p.hand, deck, discard)
  # draw two cards !
  return skipNextTurn(index_orig, rev_incr, N), players, deck, discard

def drawFour(player_index, rev_incr, N, players, deck, discard):
  index_orig = player_index # saves this so we can use func below

  # gives us the index of the player who needs to draw four cards
  p = players[nextTurn(player_index, rev_incr, N)]
  added_card, p.hand, deck, discard = drawCard(p.hand, deck, discard)
  added_card, p.hand, deck, discard = drawCard(p.hand, deck, discard)
  added_card, p.hand, deck, discard = drawCard(p.hand, deck, discard)
  added_card, p.hand, deck, discard = drawCard(p.hand, deck, discard)
  # draw four cards !
  return skipNextTurn(index_orig, rev_incr, N), players, deck, discard

def determineWildColor(player):
  # randomly picks a color out of the max # of colored cards in that player's hand

  hand = player.hand
  red_count = 0
  blue_count = 0
  green_count = 0
  yellow_count = 0
  for card in hand:
    if card.color == "Red":
      red_count += 1
    elif card.color == "Blue":
      blue_count += 1
    elif card.color == "Green":
      green_count += 1
    elif card.color == "Yellow":
      yellow_count += 1
  
  colors = ["Red", "Blue", "Green", "Yellow"]
  color_list = [red_count, blue_count, green_count, yellow_count]
  max_indices = [index for index, item in enumerate(color_list) if item == max(color_list)]

  max_indices = np.array(max_indices)
  np.random.shuffle(max_indices)

  return colors[max_indices[0]]


In [ ]:
def playGame(players, deck, discard):
  #print("Starting Game...")

  color_chosen = ""
  turn_count = 0
  player_index = 0
  rev_incr = 1 # turn order starts forward
  N = players.size  # number of players
  running = True    # boolean variable to keep the game running
  wild_played = False

  # the order of the players array is our turn count for the game.
  # if a reverse card is played, we will reverse this array.

  while running:
    while player_index < N:
      player = players[player_index]
      turn_count += 1
      #print("Turn: ", turn_count)
      curr_card = discard[len(discard) - 1]   # (index for discard array to always grab first card on top)


      playable_colors, black_cards = findPlayable(player.hand, curr_card, wild_played, color_chosen)

      # include check here if player cannot play ANY cards
      if len(playable_colors) == 0 and len(black_cards) == 0:
        #print("No playable cards for Player "+ str(player.num) +"! Drawing...")
        drawn_card, player.hand, deck, discard = drawCard(player.hand, deck, discard)
        if drawn_card.value == curr_card.value or drawn_card.color == curr_card.color or drawn_card.color == "Black":
          # drawn card is playable, the player must play it.
          player.hand = np.delete(player.hand, findIndex(drawn_card, player.hand)) # remove card from player's hand
          played_card = drawn_card
        else:
          # drawn card was NOT playable, player's turn gets skipped.
          player_index = nextTurn(player_index, rev_incr, N)
          continue
      elif len(playable_colors) == 0:
        # a black card can be played.
        played_card = playRandomCard(black_cards)
        player.hand = np.delete(player.hand, findIndex(played_card, player.hand)) # remove card from player's hand
      else:
        # play a colored card.
        # added line: does not prioritize black or non black********************
        test_array = np.concatenate((playable_colors, black_cards), axis = 0)
        played_card = playRandomCard(test_array)
        #************************
        #played_card = playRandomCard(playable_colors)
        player.hand = np.delete(player.hand, findIndex(played_card, player.hand)) # remove card from player's hand
    
      # ********if we get to this point in the loop, a card has been played.
      wild_played = False

      discard.append(played_card)  # add card to the discard deck

      #print("Player " + str(player.num) + " played a: ", end="")
      #printCard(played_card)

      # ****************action card checks
      if played_card.value == "Skip":
        # return the index of the skipped guy so i can say who got skipped.
        skipped_index = nextTurn(player_index, rev_incr, N)
        #print("Player " + str(players[skipped_index].num) + " got skipped." )
        player_index = skipNextTurn(player_index, rev_incr, N)
        continue
      elif played_card.value == "Reverse":
        #print("REVERSE")
        rev_incr *= -1 # switch the turn order
      elif played_card.value == "Draw2":
        # next player needs to draw 2 and get their turn skipped.
        #print("draw2 was played!!!")
        skipped_index = nextTurn(player_index, rev_incr, N)
        #print("Player " + str(players[skipped_index].num) + " drew 2 and got skipped." )
        player_index, players, deck, discard = drawTwo(player_index, rev_incr, N, players, deck, discard)
        continue
      elif played_card.value == "Wild":
        color_chosen = determineWildColor(player)
        #print("Player " + str(player.num) + ": \"I choose... " + color_chosen + ".\"")
        wild_played = True
      elif played_card.value == "Draw4":
        skipped_index = nextTurn(player_index, rev_incr, N)
        #print("Player " + str(players[skipped_index].num) + " drew 4 and got skipped." )

        color_chosen = determineWildColor(player)
        #print("Player " + str(player.num) + ": \"I choose... " + color_chosen + ".\"")
        wild_played = True
        player_index, players, deck, discard = drawFour(player_index, rev_incr, N, players, deck, discard)
        continue

      #if len(player.hand) == 1:
        #print("Player " + str(player.num) + ": \"UNO!\"")
      if len(player.hand) == 0:
        #print("Player " + str(player.num) + ": \"I WINNN!\"")
        winning_player = player
        running = False # make this False when a player wins!
        break

      player_index = nextTurn(player_index, rev_incr, N)

    #printPlayers(players) # for debugging
    #print()
  #print('Player '+ str(winning_player.num)+' won!')
  return (winning_player.num)

In [ ]:
seed = 123


np.random.seed(seed)
players, deck, discard = initGame(4)
playGame(players, deck, discard)

3

In [ ]:
def monteCarlo(sample_size, runs, N=4):
  run_count = 0
  sample_count = 0
  player_wins = np.zeros(N) # index 0: Player 1; index 1: Player 2; etc.
  player_winrates = np.zeros(N)
  wins = np.zeros((runs, N))
  while run_count < runs:
    sample_count = 0
    while sample_count < sample_size:
      #np.random.seed(seed)
      players, deck, discard = initGame(N)
      winner = playGame(players, deck, discard)
      wins[run_count][winner - 1] += 1
      sample_count += 1
      #player_winrates = player_wins/sample_size
    #winrates[run_count] = player_wins
    run_count += 1
    
  #print(wins)
  return wins, sample_size

In [ ]:
def monteCarloFixed(sample_size, runs, N=4):
  run_count = 0
  sample_count = 0
  player_wins = np.zeros(N) # index 0: Player 1; index 1: Player 2; etc.
  player_winrates = np.zeros(N)
  wins = np.zeros((runs, N))
  while run_count < runs:
    sample_count = 0
    while sample_count < sample_size:
      seed = 123
      np.random.seed(seed)
      players, deck, discard = initGameFixedHands(4)
      seed = np.random.randint(2**31-1)
      np.random.seed(seed)
      winner = playGame(players, deck, discard)
      print(winner)
      wins[run_count][winner - 1] += 1
      sample_count += 1
    run_count += 1
    
  print(wins)
  return wins, sample_size

In [ ]:
def histoBox(wr, player_num, sample_size):    
    w = 1
    plt.hist(wr, bins=np.arange(min(wr), max(wr) + w, w))
    #plt.title("1D Histogram of Player "+str(player_num)+"'s Winrate")
    plt.xlabel("Number of Wins of out of " + str(sample_size))
    plt.ylabel("Frequency")
    plt.show()